In [ ]:
import numpy as np
import pandas as pd
import random
import math
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
import matplotlib.pyplot as plt

In [ ]:
class AIS:
    def __init__(self, data, abNum, gama, mabNum,matingRate,variationRate,max_epoch):
        self.cityNums = data.shape[0] # 城市数
        self.max_epoch = max_epoch #最大迭代次数
        self.dist = squareform(pdist(data, metric='euclidean'))  # 计算N个城市的距离矩阵
        self.abNum = abNum   # 抗体数
        self.mabNum = mabNum # 记忆细胞数
        self.matingRate = matingRate   
        self.variationRate = variationRate
        Abs = list()
        for i in range(abNum): # 随机生成abNum个抗体         
            temp = random.sample(range(self.cityNums), self.cityNums)
            Abs.append(temp)
        self.Abs = np.array(Abs) # 抗体
        self.affAgAb = - np.ones(abNum) * np.inf
        self.affAbAb = - np.ones((abNum, abNum)) * np.inf
        self.density = np.zeros(abNum)
        self.gama = gama   # 抗体-抗体间亲和力阈值
        self.memory = self.Abs[random.sample(range(abNum), mabNum)]
        self.bestAb = None
        return
    
    def calcu_affAgAb(self): #计算抗原与抗体间的亲和度
        for i in range(self.abNum):
            fitValue = 0
            for c in range(self.cityNums-1):
                fitValue += self.dist[self.Abs[i,c], self.Abs[i,c+1]]
            self.affAgAb[i] = 1 / fitValue
        return 
    
    def calcu_affAbAb(self):  #计算抗体与抗体间亲和度
        for i in range(self.abNum):
            for j in range(self.abNum):
                if i != j:
                    count = 0
                    for k in range(self.cityNums):
                        if self.Abs[i][k] != self.Abs[j][k]:
                            count += 1
                    self.affAbAb[i,j] = 1/(1+count)
        return
    
    def calcu_density(self):  #计算抗体密度
        for i in range(self.abNum):
            count1 = 0
            for j in range(self.abNum):
                count2 = 0
                if i != j and self.affAbAb[i,j] > self.gama:
                    count2 += 1
                count1 += count2
            self.density[i] = count1/self.abNum
        return
    
    def maxAffAbAbInMemory(self, ab):  #获取在记忆细胞中，与当前抗体亲和度最大的细胞
        maxSubscript = 0
        maxAff = -np.inf
        for i,m in enumerate(self.memory):
            count = 0
            for k in range(self.cityNums):      
                if m[k] != ab[k]:
                    count += 1
                aff = 1/(1+count)
                if aff > maxAff:
                    maxAff = aff
                    maxSubscript = i
        return maxSubscript
    
    def in_memory(self):   # 成为记忆细胞
        maxSubscript = np.argmax(self.affAgAb)  #与抗原亲和力最大的抗体
        self.bestAb = maxSubscript
        ab = self.Abs[maxSubscript]
        repace = self.maxAffAbAbInMemory(ab) #变为记忆细胞
        self.memory[repace] = ab
        return
    
    def getBestInMemory(self):  #获取记忆细胞中的最优
        shortest = np.inf
        index = None
        for i,m in enumerate(self.memory):
            sumdis = 0
            for c in range(self.cityNums-1):
                sumdis += self.dist[m[c], m[c+1]]
            if sumdis < shortest:
                shortest = sumdis
                index = i
        return shortest, self.memory[index]
    
    def newAbs(self):
        m1 = np.mean(self.affAgAb)  #抗体-抗原亲和度的均值
        m2 = np.mean(self.density)  #抗体密度的均值
        goodAb = list()
        for i,ab in enumerate(self.Abs):
            if i == self.bestAb:
                goodAb.append(ab)
                continue   # 当前最优解无条件保留
            if self.affAgAb[i] < m1 and self.density[i] > m2:  #高亲和度低密度的抗体得以保留
                goodAb.append(ab)
        lenGoodAb = len(goodAb)
        diff = self.abNum - lenGoodAb
        if diff > 0 and lenGoodAb > 0:
            # 可选遗传算法的算子 ---变异、交配
            for i in range(diff):
                index = random.randint(0,lenGoodAb-1)
                goodAb.append(goodAb[index])
            self.Abs = np.array(goodAb)
        self.mating()
        self.variation()
        return 
    
    def mating(self):   # 有序交叉法
        willmate = list()
        for k, ab in enumerate(self.Abs):
            if k == self.bestAb:
                continue # 当前最优解不交配
            r = random.random()
            if r < self.matingRate:
                willmate.append(ab)
        if len(willmate) >= 2 : # 交配个体大于2才进行本轮交配
            if len(willmate) % 2 != 0:  # 交配个体为基数
                delIndex = random.randint(0,len(willmate)-1)  #随机剔除一个
                del willmate[delIndex]
            matingMap = random.sample(range(len(willmate)), len(willmate))
            for i in range(0, len(matingMap), 2):  # 有序交叉 交配过程
                x1 = matingMap[i]
                x2 = matingMap[i+1]
                positions = random.sample(range(self.cityNums), 2)  # 随机两个交叉位
                positions.sort()
                com1  = list(willmate[x1][positions[0]:positions[1]+1])
                com2  = list(willmate[x2][positions[0]:positions[1]+1])
                limit = list(range(positions[0])) + list(range(positions[1]+1, self.cityNums))
                for p in limit:            
                    temp = willmate[x1][p]
                    if willmate[x2][p] not in com1:
                        willmate[x1][p] = willmate[x2][p]
                    else:
                        index = com1.index(willmate[x2][p])
                        while com2[index] in com1:
                            index = com1.index(com2[index])
                        willmate[x1][p] = com2[index]
                    if temp not in com2:
                        willmate[x2][p] = temp
                    else:
                        index = com2.index(temp)
                        while com1[index] in com2:
                            index = com2.index(com1[index])
                        willmate[x2][p] = com1[index]
        return
    
    def variation(self):  # 倒置变异
        for k, ab in enumerate(self.Abs):
            if k == self.bestAb:
                continue # 当前最优解不变异
            r = random.random()
            if r < self.variationRate:
                positions = random.sample(range(self.cityNums), 2) # 随机两个变异
                positions.sort()
                diff = (positions[1] - positions[0])/2
                for i in range(positions[0], math.ceil(diff+positions[0])):  # 倒置操作
                    temp = ab[i]
                    ab[i] = ab[int(i+(positions[0]+diff-i)*2)]
                    ab[int(i+(positions[0]+diff-i)*2)] = temp
        return 
    
    def run(self):
        t = 0
        best = np.inf
        bestRoute = None
        while t < self.max_epoch:
            t += 1
            self.calcu_affAgAb()
            self.calcu_affAbAb()
            self.calcu_density()
            self.in_memory()
            self.newAbs()
            shortest, route = self.getBestInMemory()
            '''
            if abs(shortest-best) < 1e-12:
                break
            else:
                best = shortest
                bestRoute = route
            '''
            best = shortest
            bestRoute = route
        print('迭代次数:',str(t))
        print('最优值:',str(best))
        print('最优解:',bestRoute)
        return 

In [ ]:
data = np.array(pd.read_excel("../dataSet/cities.xlsx",header=None))
abNum = 100
gama = 0.0001
mabNum = 5
matingRate    = 0.9   #抗体交配概率
variationRate = 0.01  #抗体变异概率
max_epoch = 100
ais = AIS(data,abNum,gama,mabNum,matingRate,variationRate,max_epoch)
ais.run()